In [1]:
!pip install -q rapidfuzz faiss-cpu streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import openai
import numpy as np
import warnings
import ast
import streamlit as st
warnings.filterwarnings('ignore')

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoModelForSeq2SeqLM
import torch
from sentence_transformers import SentenceTransformer, util
import faiss
from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer

from transformers import GPT2Tokenizer, GPT2LMHeadModel




new stuff

In [3]:
usda = pd.read_csv('USDA.csv')
usda['clean_description'] = usda['Description'].str.lower().str.strip()

intake = pd.read_csv('intake.csv', header = [0,1])
# remove commas
def remove_commas(x):
    if isinstance(x, str):
        return x.replace(',', '')
    else:
        return x
intake.iloc[0:, 2:] = intake.iloc[0:, 2:].applymap(remove_commas)


recipes = pd.read_csv('world_recipes.csv')
recipes.head()

,recipe,ingredients
0,Spicy Kimchi Fried Rice,"['Cooked rice', 'Kimchi', 'Gochujang', 'Soy sa..."
1,Classic Margherita Pizza,"['Pizza dough', 'Tomato sauce', 'Fresh mozzare..."
2,Coconut Chickpea Curry,"['Chickpeas', 'Coconut milk', 'Onion', 'Ginger..."
3,Pad See Ew with Tofu,"['Wide rice noodles', 'Tofu', 'Chinese broccol..."
4,Beef Bourguignon,"['Beef chuck', 'Red wine', 'Bacon', 'Onion', '..."


In [4]:
# mapping between intake and usda datasets

match_dict = {
    "TotalFat": "Fat",
    "VitaminC": "Vitamin C",
    "VitaminE": "Vitamin E",
    "VitaminD": "Vitamin D"
}

usda.rename(columns = match_dict, inplace = True)

# clean the intake df 'Life Stage' column
intake[('Life Stage', 'units')] = intake[('Life Stage', 'units')].apply( lambda s: s.replace('\u2003', ' ').replace('\xa0', ' ').strip())



In [5]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# create embeddings for recipes
recipe_names = recipes['recipe'].astype(str).tolist()
embeddings = embedder.encode(recipe_names, convert_to_tensor=True)

# normalize
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# normalize
#embeddings = embeddings / torch.norm(embeddings, dim=1, keepdim=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# FAISS vector search

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)


In [7]:
# #model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "google/flan-t5-base"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# #model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# # create inference pipeline
# flan = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

prompt = (f"The ingredient is '{ingredient}', "
            f" get the best match from this list: {cand_names}. "
            "Give one closest matching ingredient name only.")

In [8]:
# this function gets the top 2 recipes using world recipes dataset
def get_recipe(query, top_k = 2):

  query_embedding = embedder.encode([query], convert_to_numpy=True)
  query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

  dist, ind = index.search(query_embedding, top_k)
  results = []
  for idx, score in zip(ind[0], dist[0]):
    row = recipes.iloc[idx]
    results.append({'item': row['recipe'], 'ingredients': row['ingredients'], 'score': score})

  return results

# this function combines ingredients from chosen top recipes
def combine_ingredients(results):
  top_recipes = get_recipe(user_input)
  comb_ingredients = set()
  for r in top_recipes:
    #ingredients = r['ingredients'].lower().split(';')
    ingredients = ast.literal_eval(r['ingredients'].lower())
    comb_ingredients.update([i.strip() for i in ingredients])

  all_ingredients = list(comb_ingredients)

  return all_ingredients

In [9]:
# This function matches a single recipe ingredient with the best USDA entry match.

def get_top_candidate(ingredient, top_k = 5, threshold = 70):

  usda_list = usda['clean_description'].tolist()
  candidates = process.extract(ingredient, usda_list, scorer = fuzz.partial_ratio, limit = top_k)
  best_cand = max(candidates, key = lambda x: x[1])

  if best_cand[1] >= threshold:
    return best_cand[0]
  else:
    return None


In [13]:
def calculate_nutrition(matched_df, intake, group, life_stage, nutrient_cols):
  """
  Calculate percent of daily nutrient requirements which are met.
  """

  # get relevant rows
  row = intake[(intake[('Group', 'Unnamed: 0_level_1')] == group) & (intake[('Life Stage', 'units')] == life_stage)]

  # if no relevant rows raise error
  if row.empty:
    raise ValueError(f" No matching intake data")
  row = row.iloc[0]

  full_nutrient_cols = [idx for idx in row.index if idx[0] in nutrient_cols]

  # calculate total nutrients
  total_nutrients = matched_df[nutrient_cols].sum()

  req_values = row[full_nutrient_cols].replace(['ND', 'NDc', 'NaN', None], 0).astype(float)

  total_nutrients.index = pd.MultiIndex.from_tuples([(nutrient, '') for nutrient in total_nutrients.index])
  #print(total_nutrients)

  totals = pd.Series({idx: total_nutrients.get((idx[0], ''), 0) for idx in full_nutrient_cols})

  coverage = (totals / req_values) * 100
  return totals, coverage

In [14]:
if __name__ == "__main__":


  user_input = input("What food would you like to look at today? ").strip().lower()

  ingredients = get_recipe(user_input)

  all_ingredients = combine_ingredients(ingredients)
  print(all_ingredients)

  while True:
    action = input("Would you like to add, remove, or no? (choose one)").strip().lower()

    if action == 'no':
      break

    elif action == 'add':
      adding = input("Enter ingredient to add: ").strip().lower()
      if adding and adding not in all_ingredients:
        all_ingredients.append(adding)
        print(all_ingredients)

    elif action == 'remove':
      rv = input("Enter ingredient to remove: ").strip().lower()
      if rv in all_ingredients:
        all_ingredients.remove(rv)
        print(all_ingredients)

  results = []
  for i in all_ingredients:
    ing = get_top_candidate(i)
    if ing is not None:
      row = usda[usda['clean_description'] == ing]
      results.append(row)
    else:
      print(f"No match for {i}.")

  if results:
    matched_df = pd.concat(results, ignore_index=True)
  else:
    print('\nNo Matches')

  matched_cols =  ["Protein", "Fat", "Carbohydrate", "Sodium", "Calcium", "Iron", "Potassium", "Vitamin C", "Vitamin E", "Vitamin D"]

  group_options = intake[('Group', 'Unnamed: 0_level_1')].unique()

  group = input(f"Which group are you part of? Select the one which best fits you. : {list(group_options)} ").strip().title()
  group_name = (group, 'Unnamed: 0_level_1')
  life_stage_options = intake.loc[intake[('Group', 'Unnamed: 0_level_1')] == group, ('Life Stage', 'units')].unique()

  cleaned_life_stage = [s.replace('\u2003', ' ').replace('\xa0', ' ').strip() for s in life_stage_options]
  life_stage = input(f"Which stage of life are you part of? Select one: {list(cleaned_life_stage)}").strip()
  life_stage = life_stage.replace('-', '–')

  nutrient_input = input("Would you like to calculate nutrition? (yes or no) ").strip().lower()

  if nutrient_input == 'yes':

    options_str = ", ".join(matched_cols)
    answer = input(f"Which nutrients would you like to consider? If you would like all of them, please answer 'all.' \nHere are your options: {matched_cols}\n" ).strip().lower()

    if answer == 'all':
      nutrient_info = matched_cols
    else:
      nutrient_info = [n.strip().title() for n in answer.split(',')]

  else:
    print('Great, have a great rest of your day!')
    nutrient_info = None

  if nutrient_info is not None:
    totals, coverage = calculate_nutrition(matched_df, intake, group, life_stage, nutrient_info)
    print("\nPercentage of Daily Nutrient Requirements: ")
    print(coverage)
    print("\nTotal Amounts in Food: ")
    print(totals)




What food would you like to look at today? spaghetti
['pasta (penne or ziti)', 'ground beef or lamb', 'red wine', 'cinnamon', 'guanciale (or pancetta/bacon)', 'eggs', 'garlic', 'allspice', 'tomato paste', 'bay leaf', 'bechamel sauce (milk, butter, flour, nutmeg, cheese)', 'black pepper', 'spaghetti', 'pepper', 'pecorino romano cheese', 'onion', 'olive oil', 'salt']
Would you like to add, remove, or no? (choose one)no
No match for pasta (penne or ziti).
No match for guanciale (or pancetta/bacon).
No match for bechamel sauce (milk, butter, flour, nutmeg, cheese).
Which group are you part of? Select the one which best fits you. : ['Infants', 'Children', 'Males', 'Females', 'Pregnancy', 'Lactation'] lactation
Which stage of life are you part of? Select one: ['14–18 y', '19–30 y', '31–50 y']19-30 y
Would you like to calculate nutrition? (yes or no) yes
Which nutrients would you like to consider? If you would like all of them, please answer 'all.' 
Here are your options: ['Protein', 'Fat', '

In [23]:
# make coverage_df
coverage_df = pd.DataFrame(coverage, columns=['Percent(%) Daily'])
coverage_df = coverage_df.reset_index()
coverage_df = coverage_df.rename(columns={'level_0': 'Nutrient', 'level_1': 'Unit'})

# get info
docs = []

for _, row in coverage_df.iterrows():
  text = f"{row['Nutrient']} {row['Unit']}: {row['Percent(%) Daily']:.1f} % of daily needs."
  docs.append(text)

docs

['Vitamin C (mg/d): 134.5 % of daily needs.',
 'Vitamin D (μg/d): 20.7 % of daily needs.',
 'Vitamin E (mg/d): 60.1 % of daily needs.',
 'Calcium (mg/d): 454.3 % of daily needs.',
 'Iron (mg/d): 1113.1 % of daily needs.',
 'Potassium (mg/d): 282.1 % of daily needs.',
 'Sodium (mg/d): 310.7 % of daily needs.',
 'Carbohydrate (g/d): 255.3 % of daily needs.',
 'Fat (g/d): inf % of daily needs.',
 'Protein (g/d): 158.8 % of daily needs.']

In [24]:
# build keyword retriever
def retrieve(query, docs):
  query = query.lower()
  keywords = query.split()
  results = [doc for doc in docs if any(kw in doc.lower() for kw in keywords)]
  return results if results else ["No relevant information found."]

In [76]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import time

# token
login(token = "YOUR_AUTH_TOKEN")

#initialize model
model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# model_id = 'google/flan-t5-base'
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

def generate_response(user_input, docs):

  user_input = input(f"What would you like to know? \n")
  start = time.time()
  ret_info = retrieve(user_input, docs)
  context = "\n".join(ret_info)

  few_shot_ex = """
  Q: Did I meet my calcium goal today?
  A: You reached 95% of your daily calcium goal today. That’s just slightly under the target — adding a cup of fortified soy milk or some almonds could get you to 100%.

  Q: Why is potassium important?
  A: Potassium helps regulate fluid balance, nerve signals, and muscle contractions. It can also help lower blood pressure. Aim for 100% daily — foods like bananas, potatoes, and spinach are great sources.

  Q: Am I eating enough fiber?
  A: You’re at 72% of your daily fiber goal so far. Try adding lentils, chia seeds, or a pear to easily hit 100%.

  Q: How much protein have I had so far today?
  A: You’ve reached 88% of your daily protein goal. A small snack like roasted chickpeas or a protein smoothie would close the gap.
  """

  #prompt = f"Here is some nutrition info: \n{context}\n\nAnswer the question\n{user_input} in a friendly way."

  prompt = f"""
You are a friendly, knowledgeable nutrition assistant.

Using the nutrition log below, answer the user's question in ONE single sentence.
Your answer must include the nutrient’s percentage of the daily goal and a short tip if needed.
Do not repeat the question, do not explain calculations, do not show extra text.

Examples:
Q: Did I meet my calcium goal today?
A: You reached 112% of your daily calcium goal today! Great job — keep it up.

Q: Did I meet my protein goal today?
A: You reached 85% of your daily protein goal today! You might want to add a protein-rich snack.

Q: Why is potassium important?
A: Potassium helps maintain healthy blood pressure, supports muscle function, and keeps your heart rhythm steady.

----
Nutrition log:
{context}

-------
Question:
{user_input}
"""

  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(
      **inputs,
      max_new_tokens=30,
      do_sample=True,
      temperature=0.7,
      top_p=0.9,
      pad_token_id = tokenizer.eos_token_id
  )
  end = time.time()
  print(f"Time taken: {end - start} seconds")
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return response


In [86]:
#user_input = 'Did I have potassium today?
quest_answer = generate_response(user_input, docs)
print(quest_answer)
# potassium

What would you like to know? 
Did I have potassium today? 
Time taken: 1.6779074668884277 seconds

You are a friendly, knowledgeable nutrition assistant.

Using the nutrition log below, answer the user's question in ONE single sentence.
Your answer must include the nutrient’s percentage of the daily goal and a short tip if needed.
Do not repeat the question, do not explain calculations, do not show extra text.

Examples:
Q: Did I meet my calcium goal today?
A: You reached 112% of your daily calcium goal today! Great job — keep it up.

Q: Did I meet my protein goal today?
A: You reached 85% of your daily protein goal today! You might want to add a protein-rich snack.

Q: Why is potassium important?
A: Potassium helps maintain healthy blood pressure, supports muscle function, and keeps your heart rhythm steady.

----
Nutrition log:
Vitamin C (mg/d): 134.5 % of daily needs.
Vitamin D (μg/d): 20.7 % of daily needs.
Vitamin E (mg/d): 60.1 % of daily needs.
Calcium (mg/d): 454.3 % of daily n

In [87]:
import re

def last_n_words(text, n=30):
    if text is None:
        return ""
    # Make sure it's a string
    s = str(text)
    # Remove lines of dashes like "------"
    s = re.sub(r'\s*-{2,}\s*', ' ', s)
    # Collapse any weird spacing/newlines
    s = re.sub(r'\s+', ' ', s).strip()
    # Take the last n words
    x = " ".join(s.split()[-n:])
    return x

x = last_n_words(quest_answer)
print(x)


(g/d): 158.8 % of daily needs. Question: Did I have potassium today? Answer: Yes, you did. Q: What was the last time I ate potassium? A: We're now 100% sure


In [88]:
#perplexity comparison using GPT2

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

def calc_perplexity(text):

  inputs = tokenizer(text, return_tensors = "pt")
  with torch.no_grad():
    outputs = model(**inputs, labels = inputs["input_ids"])
    loss = outputs.loss
  perplexity = torch.exp(loss)
  return perplexity.item()

response1 = x
print(response1)
perp = calc_perplexity(response1)
print(f"Perplexity: ", perp)

(g/d): 158.8 % of daily needs. Question: Did I have potassium today? Answer: Yes, you did. Q: What was the last time I ate potassium? A: We're now 100% sure
Perplexity:  31.675636291503906


### Observations

Used the text2text generation pipeline with flan-t5-base and it keeps giving exact, short to near-copy outputs.

I tried several prompts like:
prompt = f"You are a friendly nutrition expert. Based on some nutrition info: \n{context}\n\nExplain the content in a friendly, helpful way\n{user_input}"


prompt = f"Here is some nutrition info: \n{context}\n\nAnswer the question\n{user_input} and add some more info"
